# Task #093.1: 高级特征工程框架

**目标**: 在 Jupyter 环境下，构建能直接对接 TimescaleDB 的特征提取框架

**核心功能**:
1. 从 TimescaleDB 载入历史数据
2. 使用 AdvancedFeatureBuilder 生成机器学习特征
3. 对 close 价格进行分数差分处理，寻找最优 d 值
4. 生成具备"记忆性与平稳性"平衡的特征集

**数据源**: AAPL.US (11,361行), TSLA.US (3,908行)

In [ ]:
# 环境初始化
import sys
sys.path.insert(0, '/opt/mt5-crs')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from numba import jit

# 导入项目模块
from src.database.timescale_client import TimescaleClient
from src.feature_engineering.advanced_features import AdvancedFeatures

# 配置
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('seaborn-v0_8-darkgrid')

print("✅ 环境初始化完成")

## Step 1: 连接 TimescaleDB 并载入数据

In [ ]:
# 初始化 TimescaleClient
client = TimescaleClient()

# 检查连接
if client.check_connection():
    print("✅ TimescaleDB 连接成功")
else:
    print("❌ TimescaleDB 连接失败")
    raise RuntimeError("无法连接到数据库")

In [ ]:
# 载入 AAPL 和 TSLA 数据
def load_symbol_data(symbol: str) -> pd.DataFrame:
    """从 TimescaleDB 载入指定股票的历史数据"""
    query = f"""
    SELECT 
        date,
        open,
        high,
        low,
        close,
        volume,
        adjusted_close
    FROM daily_bars
    WHERE symbol = '{symbol}'
    ORDER BY date ASC;
    """
    
    df = pd.read_sql(query, client.engine)
    df['symbol'] = symbol
    df['date'] = pd.to_datetime(df['date'])
    
    return df

# 载入数据
print("📊 载入历史数据...")
aapl_df = load_symbol_data('AAPL.US')
tsla_df = load_symbol_data('TSLA.US')

print(f"\n✅ 数据载入完成:")
print(f"   AAPL: {len(aapl_df)} 行")
print(f"   TSLA: {len(tsla_df)} 行")

# 显示数据预览
print("\n📋 AAPL 数据预览:")
aapl_df.head()

## Step 2: 构建 AdvancedFeatureBuilder 类

集成 numba 加速的计算算子，优化分数差分计算性能

In [ ]:
class AdvancedFeatureBuilder:
    """
    高级特征构建器
    
    集成:
    1. Numba 加速的分数差分计算
    2. ADF 平稳性测试
    3. 最优 d 值搜索
    4. 特征平稳性验证
    """
    
    @staticmethod
    @jit(nopython=True)
    def compute_frac_diff_weights(d: float, threshold: float = 1e-5, max_k: int = 100) -> np.ndarray:
        """使用 Numba 加速计算分数差分权重"""
        weights = np.zeros(max_k)
        weights[0] = 1.0
        k = 1
        
        for i in range(1, max_k):
            weight = -weights[i-1] * (d - k + 1) / k
            if abs(weight) < threshold:
                break
            weights[i] = weight
            k += 1
        
        # 返回非零权重
        return weights[:k]
    
    @staticmethod
    @jit(nopython=True)
    def apply_frac_diff(series: np.ndarray, weights: np.ndarray) -> np.ndarray:
        """使用 Numba 加速应用分数差分"""
        n = len(series)
        w_len = len(weights)
        result = np.full(n, np.nan)
        
        for i in range(w_len - 1, n):
            result[i] = np.dot(weights[::-1], series[i - w_len + 1:i + 1])
        
        return result
    
    @classmethod
    def fractional_diff_fast(cls, series: pd.Series, d: float = 0.5, threshold: float = 1e-5) -> pd.Series:
        """快速分数差分（使用 Numba 加速）"""
        weights = cls.compute_frac_diff_weights(d, threshold)
        values = series.values
        result = cls.apply_frac_diff(values, weights)
        
        return pd.Series(result, index=series.index)
    
    @staticmethod
    def adf_test(series: pd.Series, significance_level: float = 0.05) -> dict:
        """执行 ADF 平稳性测试"""
        # 去除 NaN
        series_clean = series.dropna()
        
        if len(series_clean) < 10:
            return {
                'statistic': np.nan,
                'pvalue': np.nan,
                'is_stationary': False,
                'reason': 'insufficient_data'
            }
        
        try:
            result = adfuller(series_clean, autolag='AIC')
            
            return {
                'statistic': result[0],
                'pvalue': result[1],
                'is_stationary': result[1] < significance_level,
                'critical_values': result[4],
                'reason': 'pass' if result[1] < significance_level else 'non_stationary'
            }
        except Exception as e:
            return {
                'statistic': np.nan,
                'pvalue': np.nan,
                'is_stationary': False,
                'reason': f'error: {str(e)}'
            }
    
    @classmethod
    def find_optimal_d(cls, series: pd.Series, d_range: list = None, 
                      significance_level: float = 0.05, verbose: bool = True) -> dict:
        """
        寻找最优的分数差分阶数 d
        
        目标: 找到最小的 d 值，使得序列平稳（ADF p-value < 0.05）
        """
        if d_range is None:
            d_range = np.arange(0.0, 1.1, 0.1)
        
        results = []
        
        for d in d_range:
            # 应用分数差分
            diff_series = cls.fractional_diff_fast(series, d=d)
            
            # ADF 测试
            adf_result = cls.adf_test(diff_series, significance_level)
            
            # 计算相关性（衡量记忆性保留）
            correlation = series.corr(diff_series.shift(1))
            
            results.append({
                'd': d,
                'adf_statistic': adf_result['statistic'],
                'adf_pvalue': adf_result['pvalue'],
                'is_stationary': adf_result['is_stationary'],
                'correlation': correlation
            })
            
            if verbose:
                status = "✅" if adf_result['is_stationary'] else "❌"
                print(f"{status} d={d:.2f}: p-value={adf_result['pvalue']:.4f}, corr={correlation:.4f}")
        
        results_df = pd.DataFrame(results)
        
        # 找到第一个平稳的 d 值（最小 d）
        stationary_results = results_df[results_df['is_stationary']]
        
        if len(stationary_results) > 0:
            optimal_d = stationary_results.iloc[0]['d']
            optimal_result = stationary_results.iloc[0].to_dict()
        else:
            # 如果没有平稳的，选择 p-value 最小的
            optimal_idx = results_df['adf_pvalue'].idxmin()
            optimal_d = results_df.loc[optimal_idx, 'd']
            optimal_result = results_df.loc[optimal_idx].to_dict()
        
        return {
            'optimal_d': optimal_d,
            'optimal_result': optimal_result,
            'all_results': results_df
        }
    
    @classmethod
    def build_features(cls, df: pd.DataFrame, optimal_d: float = None) -> pd.DataFrame:
        """
        构建完整的特征集
        
        如果提供 optimal_d，使用该值；否则自动搜索最优 d
        """
        print("🔧 开始构建高级特征...")
        
        # 1. 如果没有提供 optimal_d，自动搜索
        if optimal_d is None:
            print("\n🔍 搜索最优分数差分阶数 d...")
            opt_result = cls.find_optimal_d(df['close'], verbose=True)
            optimal_d = opt_result['optimal_d']
            print(f"\n✅ 最优 d 值: {optimal_d:.2f}")
        
        # 2. 使用 AdvancedFeatures 计算所有特征
        df = AdvancedFeatures.compute_all_advanced_features(df)
        
        # 3. 添加使用最优 d 的分数差分特征
        df['frac_diff_close_optimal'] = cls.fractional_diff_fast(df['close'], d=optimal_d)
        
        print(f"\n✅ 特征构建完成，共 {len(df.columns)} 个特征")
        
        return df

print("✅ AdvancedFeatureBuilder 类定义完成")

## Step 3: 寻找 AAPL 的最优 d 值

In [ ]:
# 搜索 AAPL 的最优 d 值
print("🔍 为 AAPL 搜索最优分数差分阶数 d...\n")

aapl_opt = AdvancedFeatureBuilder.find_optimal_d(
    aapl_df['close'],
    d_range=np.arange(0.0, 1.1, 0.05),
    significance_level=0.05,
    verbose=True
)

print(f"\n{'='*60}")
print(f"最优结果:")
print(f"  d 值: {aapl_opt['optimal_d']:.2f}")
print(f"  ADF p-value: {aapl_opt['optimal_result']['adf_pvalue']:.6f}")
print(f"  平稳性: {'✅ 是' if aapl_opt['optimal_result']['is_stationary'] else '❌ 否'}")
print(f"  相关性: {aapl_opt['optimal_result']['correlation']:.4f}")
print(f"{'='*60}")

In [ ]:
# 可视化不同 d 值的效果
results_df = aapl_opt['all_results']

fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# 图1: ADF p-value vs d
axes[0].plot(results_df['d'], results_df['adf_pvalue'], marker='o', linewidth=2)
axes[0].axhline(y=0.05, color='r', linestyle='--', label='显著性水平 (0.05)')
axes[0].axvline(x=aapl_opt['optimal_d'], color='g', linestyle='--', label=f"最优 d={aapl_opt['optimal_d']:.2f}")
axes[0].set_xlabel('分数差分阶数 (d)')
axes[0].set_ylabel('ADF p-value')
axes[0].set_title('ADF 平稳性测试: p-value vs d')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 图2: 相关性 vs d
axes[1].plot(results_df['d'], results_df['correlation'], marker='s', color='orange', linewidth=2)
axes[1].axvline(x=aapl_opt['optimal_d'], color='g', linestyle='--', label=f"最优 d={aapl_opt['optimal_d']:.2f}")
axes[1].set_xlabel('分数差分阶数 (d)')
axes[1].set_ylabel('与原序列的相关性')
axes[1].set_title('记忆性保留: 相关性 vs d')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/opt/mt5-crs/docs/archive/tasks/TASK_093_1/aapl_optimal_d_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ 可视化结果已保存")

## Step 4: 构建完整特征集

In [ ]:
# 为 AAPL 构建完整特征集
print("🔧 为 AAPL 构建完整特征集...\n")

aapl_features = AdvancedFeatureBuilder.build_features(
    aapl_df.copy(),
    optimal_d=aapl_opt['optimal_d']
)

# 统计特征维度
feature_cols = [col for col in aapl_features.columns if col not in ['date', 'symbol', 'open', 'high', 'low', 'close', 'volume', 'adjusted_close']]

print(f"\n📊 特征统计:")
print(f"   总特征数: {len(feature_cols)}")
print(f"   数据行数: {len(aapl_features)}")
print(f"   缺失值比例: {aapl_features[feature_cols].isnull().sum().sum() / (len(aapl_features) * len(feature_cols)) * 100:.2f}%")

# 显示部分特征
print("\n📋 特征预览:")
aapl_features[['date', 'close', 'frac_diff_close_05', 'frac_diff_close_optimal']].tail(10)

In [ ]:
# 对比原始价格 vs 分数差分序列
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# 只显示最近 500 天
plot_df = aapl_features.tail(500).copy()

# 图1: 原始收盘价
axes[0].plot(plot_df['date'], plot_df['close'], linewidth=1.5, color='blue')
axes[0].set_title('原始收盘价 (AAPL)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('价格 ($)')
axes[0].grid(True, alpha=0.3)

# 图2: d=0.5 分数差分
axes[1].plot(plot_df['date'], plot_df['frac_diff_close_05'], linewidth=1.5, color='green')
axes[1].set_title('分数差分 (d=0.5)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('差分值')
axes[1].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[1].grid(True, alpha=0.3)

# 图3: 最优 d 值分数差分
axes[2].plot(plot_df['date'], plot_df['frac_diff_close_optimal'], linewidth=1.5, color='orange')
axes[2].set_title(f'分数差分 (最优 d={aapl_opt["optimal_d"]:.2f})', fontsize=12, fontweight='bold')
axes[2].set_xlabel('日期')
axes[2].set_ylabel('差分值')
axes[2].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/opt/mt5-crs/docs/archive/tasks/TASK_093_1/aapl_fractional_diff_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ 对比图已保存")

## Step 5: 平稳性验证

In [ ]:
# 验证主要特征的平稳性
print("🔍 验证特征平稳性...\n")

test_features = [
    'close',
    'frac_diff_close_05',
    'frac_diff_close_07',
    'frac_diff_close_optimal',
    'frac_diff_volume_05',
]

stationarity_results = []

for feature in test_features:
    if feature in aapl_features.columns:
        result = AdvancedFeatureBuilder.adf_test(aapl_features[feature])
        stationarity_results.append({
            'feature': feature,
            'adf_statistic': result['statistic'],
            'p_value': result['pvalue'],
            'is_stationary': result['is_stationary']
        })
        
        status = "✅" if result['is_stationary'] else "❌"
        print(f"{status} {feature:30s}: p-value={result['pvalue']:.6f}")

stationarity_df = pd.DataFrame(stationarity_results)
print(f"\n平稳特征比例: {stationarity_df['is_stationary'].sum() / len(stationarity_df) * 100:.1f}%")

## Step 6: 保存结果

In [ ]:
# 保存特征数据到 CSV
output_path = '/opt/mt5-crs/docs/archive/tasks/TASK_093_1/aapl_features.csv'
aapl_features.to_csv(output_path, index=False)
print(f"✅ 特征数据已保存: {output_path}")

# 保存最优 d 值结果
import json
with open('/opt/mt5-crs/docs/archive/tasks/TASK_093_1/optimal_d_result.json', 'w') as f:
    json.dump({
        'symbol': 'AAPL.US',
        'optimal_d': float(aapl_opt['optimal_d']),
        'adf_pvalue': float(aapl_opt['optimal_result']['adf_pvalue']),
        'is_stationary': bool(aapl_opt['optimal_result']['is_stationary']),
        'correlation': float(aapl_opt['optimal_result']['correlation'])
    }, f, indent=2)
print(f"✅ 最优 d 值结果已保存")

print("\n" + "="*60)
print("🎉 Task #093.1 特征工程完成!")
print("="*60)